In [1]:
# Import necessary libraries
import xarray as xr
import dask.array as da
from dask.distributed import Client, LocalCluster, progress
from dask import delayed
import h5py
import numpy as np

#import earthaccess

In [2]:
# Set up a Dask cluster with 50 workers

# Automatically detect available vCPUs
import multiprocessing
n_workers = multiprocessing.cpu_count()
print(n_workers)

128


In [4]:

# Create a LocalCluster with as many workers as vCPUs

#cluster = LocalCluster(n_workers=16, threads_per_worker=8)
#client = Client(cluster)

#print(cluster)
#cluster = LocalCluster(n_workers=50, threads_per_worker=1)
client = Client('tcp://127.0.0.1:36921')

In [5]:
########################################################################################
## Switch the function to this. See how long it takes to rescale a single granule, then see how long it 
## takes to downscale all ~7000 granules
########################################################################################
def downscaling_mursst(fns,s3sys):
    """
    This function reads a netCDF file from MUR SST 1km data,
    processes the data by masking invalid values, reshaping, and computing the mean
    temperature for each 1x1 degree box, and then returns the processed SST data.

    Parameters
    ----------
    fn : str, s3 path

    Returns
    -------
    sst : numpy.ma.core.MaskedArray (180x360)
        A masked array containing the processed SST data in a 1x1 degree resolution, with invalid values masked
        and the mean temperature computed for each granule.

    Dependencies
    ------------
    This function depends on the following libraries:
    - xarray (imported as xr)
    - numpy (imported as np)
    - s3fs (imported as s3sys)

    Notes
    -----
    The input file is expected to be in netCDF format and should contain a variable
    named 'analysed_sst' representing the sea surface temperature data, common in GHRSST products.

    The function assumes a specific data structure, with dimensions of 1x17900x36000 in the input and 
    180x360 in the output. The 17900x36000 array was first expanded to 18000x36000 using np.r_[sst[0,:],sst].
    """
    for fn in fns[:1]:
        d = xr.open_dataset(s3sys.open(fn), engine='h5netcdf')
        sst = d['analysed_sst'][0, ...]; del d
        sst0 = np.ma.masked_invalid(np.r_[sst[0:1, :], sst]); del sst
        sst = sst0.reshape(180, 100, 360, 100).mean(axis=-1).mean(axis=1); del sst0
        
    return

In [2]:
def init_S3FileSystem():
    """
    This routine automatically pull your EDL crediential from .netrc file and use it to obtain an AWS S3 credential through a podaac service accessable at https://archive.podaac.earthdata.nasa.gov/s3credentials
    
    Return:
    =======
    
    s3: an AWS S3 filesystem
    """
    import requests,s3fs
    creds = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()
    s3 = s3fs.S3FileSystem(anon=False,
                           key=creds['accessKeyId'],
                           secret=creds['secretAccessKey'], 
                           token=creds['sessionToken'])
    return s3
# Define a function to load a single granule given year, month, and day
def c_granule(fn):
    ds = h5py.File(s3sys.open(fn),'r')
    
    # Define the California Current bounding box
    min_lat, max_lat = 30, 50
    min_lon, max_lon = -140, -110
    lat,lon=ds['lat'][:],ds['lon'][:]
    i0=np.where(np.abs(lat-min_lat)==np.abs(lat-min_lat).min())[0][0]
    i1=np.where(np.abs(lat-max_lat)==np.abs(lat-max_lat).min())[0][0]
    j0=np.where(np.abs(lon-min_lon)==np.abs(lon-min_lon).min())[0][0]
    j1=np.where(np.abs(lon-max_lon)==np.abs(lon-max_lon).min())[0][0]
    
    # Define a function to process a single granule and return the regional mean SST
    #msk=ds['mask'][0,10000:11000,10000:11000]==1
    #sst=ds["analysed_sst"][0,j0:j1,i0:i1]*0.001
    sst=ds["analysed_sst"][0,:10,:10]*0.001
    #msk=ds['mask'][0,j0:j1,i0:i1]!=1
    #sst = np.ma.masked_array(sst,mask=msk).mean()
    del ds
    return sst.mean()

In [3]:
%%time
s3sys=init_S3FileSystem()
s3path="s3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/"
fns=s3sys.glob(s3path+"*.nc")
print("total granules = ",len(fns))
print("Example filename: ", fns[0])


total granules =  7602
Example filename:  podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
CPU times: user 1.46 s, sys: 37.2 ms, total: 1.49 s
Wall time: 8.42 s


In [5]:
%%time

d=h5py.File(s3sys.open('s3://'+fns[0],'rb'))
print((d.keys()))
d['analysed_sst'][:].shape

<KeysViewHDF5 ['analysed_sst', 'analysis_error', 'lat', 'lon', 'mask', 'sea_ice_fraction', 'time']>
CPU times: user 5.28 s, sys: 671 ms, total: 5.96 s
Wall time: 19.6 s


(1, 17999, 36000)

In [11]:
%%time
# Wrap the process_granule function with dask.delayed
delayed_process_granule = delayed(c_granule)
# Process all granules in parallel using Dask
results = [delayed_process_granule(fn) for fn in fns]

CPU times: user 143 ms, sys: 42.8 ms, total: 186 ms
Wall time: 162 ms


In [12]:
%%time

sst=da.compute(*results)

print(sst)

(-32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001, -32.76800000000001,

In [ ]:
sst=d['analysed_sst'][0,::100,::100]

In [ ]:
import pylab as plt
plt.imshow(sst[::-1,:]*0.001)
plt.colorbar()
print(d['